In [1]:
from google.colab import drive
import json
from openai import AzureOpenAI
import os, requests, bs4

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Upload API keys
with open('/content/drive/MyDrive/secrets/api_keys.json', encoding='utf-8-sig') as f:
    secrets = json.load(f)

os.environ["SERPAPI_API_KEY"] = secrets["SERPAPI_API_KEY"]
os.environ["AZURE_OPENAI_API_KEY"] = secrets["AZURE_OPENAI_API_KEY"]

AZURE_OPENAI_ENDPOINT = secrets["AZURE_OPENAI_ENDPOINT"]
AZURE_OPENAI_DEPLOYMENT = secrets["AZURE_OPENAI_DEPLOYMENT"]

In [2]:
def get_google_links_direct(query, num_results=3):
    api_key = os.getenv("SERPAPI_API_KEY")
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
    }
    response = requests.get("https://serpapi.com/search", params=params)
    data = response.json()
    results = data.get("organic_results", [])
    links = [r['link'] for r in results[:num_results]]
    return links

# 🔍 1. Tender search
tender_query = "security camera procurement tender site:europa.eu"
tender_links = get_google_links_direct(tender_query)
print("🔗 Tender links:", tender_links)

# 🛒 2. Supplier search
supplier_query = "security camera supplier Europe price range site:.com OR site:.co.uk"
supplier_links = get_google_links_direct(supplier_query)
print("🛒 Supplier Links:", supplier_links)

# 🔄 3. Merge All
all_links = tender_links + supplier_links
print("🔗 All links:", all_links)

🔗 Tender links: ['https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/dba52083-8af6-4ab4-82a2-d7192aa0a0f7', 'https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/37a10779-0375-4408-9236-afe3bcd36f6a', 'https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/57ee5945-6bd0-4ebf-a032-fb14900678bb']
🛒 Supplier Links: ['https://uk.swann.com/products/?srsltid=AfmBOopChlfgI8r8CFY2R_v9a9QlRGxWW9tGsyy0EFSXs3VNPdgdpclA', 'https://www.eufy.com/collections/security-camera', 'https://www.kentfaith.com/KF50.0006AEU_outdoor-security-camera-2k-wireless-camera-for-home-security-2-4ghz-wifi-security-camera-with-auto-tracking-motion-detection-with-spotlight-color-night-vision-camera-2-way-audio-euro-gauge?srsltid=AfmBOoqgLrty_CZWoTQSocAhtoRvdl0RRuVRkMHiZHZkmT5NLoj_cEmQ']
🔗 All links: ['https://op.europa.eu/en/web/public-procurement/procurement-details/-/procurement/dba52083-8af6-4ab4-82a2-d7192aa0a0f7', 'https://op.europa.

In [3]:
def fetch_page_text(url):
    try:
        res = requests.get(url, timeout=10)
        soup = bs4.BeautifulSoup(res.text, "html.parser")
        return soup.get_text(separator=" ", strip=True)
    except Exception as e:
        print("Fetch error:", e)
        return ""

tender_texts   = [fetch_page_text(u) for u in tender_links]
supplier_texts = [fetch_page_text(u) for u in supplier_links]



In [4]:

# Start Azure OpenAI Client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# Prompts
tender_prompt = f"""
You are a public tender expert for Dublin City Council.

Below are documents from security camera public tenders:
{tender_texts[0][:1500]}
{tender_texts[1][:1500] if len(tender_texts) > 1 else ''}

1. Summarize the most common **technical and legal requirements**.
2. Generate 5 clarification questions that a city official should answer before drafting a new tender.
"""

supplier_prompt = f"""
You are a market analyst advising a municipality about camera system purchases.

Below are supplier websites or documents:
{supplier_texts[0][:1500]}
{supplier_texts[1][:1500] if len(supplier_texts) > 1 else ''}

1. Summarize the most common **features, pricing, and delivery models** offered in the market.
2. Generate 5 clarifying questions to help compare or choose between supplier options.
"""

# Get the Answers
tender_resp = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT,
    messages=[{"role": "user", "content": tender_prompt}]
)

supplier_resp = client.chat.completions.create(
    model=AZURE_OPENAI_DEPLOYMENT,
    messages=[{"role": "user", "content": supplier_prompt}]
)

# Print Results
print("📑 Tender Summary + Questions:\n")
print(tender_resp.choices[0].message.content)

print("\n🛒 Supplier Summary + Questions:\n")
print(supplier_resp.choices[0].message.content)

📑 Tender Summary + Questions:

### Summary of Common **Technical and Legal Requirements** for Video Surveillance Public Tenders:

#### **Technical Requirements**
1. **Camera Types and Standards**: Digital cameras (IP-based) capable of high-resolution video recording (e.g., 1080p, 4K). These cameras must support modern image processing features such as motion detection, infra-red/night vision, wide dynamic range (WDR), weatherproof and vandal-resistant casing.
2. **Integration and Compatibility**: Cameras must integrate with existing surveillance management systems (SMS). Compatibility with widely used video formats, open protocols (ONVIF), and third-party software is essential.
3. **Network Communication**: Cameras should support secure networking protocols such as HTTPS encryption and be equipped with PoE (Power over Ethernet) for simplified cabling.
4. **Storage and Retention**: Sufficient storage solutions, typically through cloud systems or local recording servers, must meet GDPR-c

In [5]:
# -----------------------------
#  EXAMPLE ANSWERS
# -----------------------------
user_answers = """
TENDER-SIDE CLARIFICATION ANSWERS
1. Purpose & Scope
   • Main goals: crime prevention around parks and municipal buildings, plus traffic-flow monitoring at busy intersections.
   • Coverage: 10 public parks, 5 key intersections, 3 municipal buildings (≈60 cameras total).

2. Data & Privacy
   • Footage kept 30 days, encrypted at rest and in transit.
   • GDPR Article 35 DPIA will be completed; local signage required under DCC by-laws.

3. Integration & Scalability
   • Must integrate with existing 25 analogue-to-IP converted cameras and the city’s traffic-control VMS.
   • Scalable to 100 additional devices within 5 years.

4. Performance & Analytics
   • Required uptime 99.5 % monthly; SLA response ≤4 h, fix ≤24 h.
   • Mandatory analytics: motion detection + licence-plate recognition (LPR).
   • Facial recognition optional.

5. Environment & Budget
   • Energy-efficiency class A or equivalent; low-power PoE preferred.
   • Budget ceiling €120 000 (ex-VAT). Vendors may propose cost-saving options.

SUPPLIER-SIDE CLARIFICATION ANSWERS
1. Durability / Warranty
   • Outdoor lifespan ≥7 years; baseline 3-year full-replacement warranty, extendable to 5 years.

2. Integration & Firmware
   • ONVIF Profile S + RTSP; open SDK; OTA firmware upgrades required.

3. Power / Connectivity
   • Solar-powered units include battery backup ≥72 h; data plan budget 5 GB per camera / month.

4. Installation & Maintenance
   • Turn-key install by supplier; training for city technicians.
   • Annual maintenance cost cap €50 per camera.

5. Scalability & Stock
   • Volume discount 10 % for orders >50 units.
   • Supplier guarantees part availability for 5 years.
"""

# ---------------------------------------------
#  GPT-4o PROMPT for writing R&S draft
# ---------------------------------------------
draft_prompt = f"""
You are a public procurement expert working at Dublin City Council.
You have conducted a market analysis on municipal surveillance camera systems and identified both common supplier offerings and typical tender requirements.

Now write a **complete and professional 'Requirements and Specifications' section** for a new security camera tender.
Structure it just like the real Irish tender documents provided before. Include the following:

- A short introduction / background
- Detailed technical requirements in bullet points or tables (e.g., resolution, night vision, weatherproofing, storage, analytics)
- Legal and compliance requirements (especially GDPR and EU procurement directives)
- Installation and maintenance responsibilities
- Service levels (SLAs), warranties, and support
- Any environmental or sustainability standards
- Pricing and contractual expectations

Make sure the structure and language matches what is commonly used in Irish public sector tenders.
Do not generate questions. Generate a final tender section text.
Length: at least 700–1000 words.


--- USER NOTES ---
{user_answers}
"""

draft_resp = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a public procurement expert at Dublin City Council, drafting professional tender documentation."},
        {"role": "user", "content": draft_prompt}
    ],
    temperature=0.2,
    max_tokens=3000,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
)

print("📄 FINAL REQUIREMENTS & SPECIFICATIONS\n")
print(draft_resp.choices[0].message.content)


📄 FINAL REQUIREMENTS & SPECIFICATIONS

### Section 3: Requirements and Specifications

#### 3.1 Introduction and Background
Dublin City Council (DCC) is seeking to procure a comprehensive municipal surveillance camera system to enhance public safety, prevent crime, and improve traffic management across the city. The system will be deployed across 10 public parks, 5 key intersections, and 3 municipal buildings, with an estimated total of 60 cameras. The solution must integrate seamlessly with the city’s existing infrastructure, including 25 analogue-to-IP converted cameras and the traffic-control Variable Message Sign (VMS) system, while being scalable to accommodate up to 100 additional devices over the next five years.

This procurement aligns with DCC’s commitment to ensuring public safety, data privacy, and environmental sustainability, while adhering to all relevant legal and regulatory requirements. The successful tenderer will be responsible for the supply, installation, commissi

In [6]:
from google.colab import files
uploaded = files.upload()  # choose PDF's

Saving Car Park Management Solution ITT_Final.pdf to Car Park Management Solution ITT_Final.pdf
Saving Final Gully Monitoring CFT.pdf to Final Gully Monitoring CFT.pdf
Saving RFT for Service Design.pdf to RFT for Service Design.pdf


In [7]:
!pip install PyPDF2


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.8 MB/s eta 0:00:00


In [8]:
from PyPDF2 import PdfReader

tender_texts = []
for filename in uploaded.keys():
    reader = PdfReader(filename)
    text = ""
    for page in reader.pages:
        page_text = page.extract_text()
        if page_text:
            text += page_text
    tender_texts.append(text)


In [9]:
draft_prompt = f"""
You are a public procurement expert working at Dublin City Council.

Here are real tender excerpts from previous Irish municipal projects:
{tender_texts[0][:1500]}
{tender_texts[1][:1500] if len(tender_texts) > 1 else ''}

You have also conducted a market analysis and collected key answers:

--- USER CLARIFICATIONS ---
{user_answers}

Now write a **comprehensive and professional 'Requirements and Specifications' section** for a new security camera tender.

Structure it similarly to the real Irish tender documents above.
Include all key elements:
- Background and purpose
- Technical specifications (preferably in table or bullet format)
- Legal and compliance (GDPR, CE, ISO, etc.)
- Installation & maintenance
- SLAs, warranty, support
- Environmental standards
- Budget & contractual terms

Style must be formal, clear, and aligned with EU procurement documentation.
Do not generate questions — only the full section text, 800–1000 words.
"""

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a tender documentation expert in a public institution."},
        {"role": "user", "content": draft_prompt}
    ],
    temperature=0.2,
    max_tokens=3500
)

print("📄 FINAL R&S SECTION:\n")
print(response.choices[0].message.content)


📄 FINAL R&S SECTION:

**REQUIREMENTS AND SPECIFICATIONS**  

**1. Background and Purpose**  
Dublin City Council (DCC) is seeking to procure a comprehensive security camera solution to enhance public safety, monitor traffic flow, and prevent crime in key municipal areas. The scope of this tender includes the supply, installation, integration, and maintenance of security cameras across 10 public parks, 5 key intersections, and 3 municipal buildings, with an estimated total of 60 cameras. The solution must integrate seamlessly with DCC’s existing infrastructure, including 25 analogue-to-IP converted cameras and the city’s traffic-control Variable Message Sign (VMS) system.  

The primary objectives of this project are:  
- Crime prevention in public parks and municipal buildings.  
- Traffic-flow monitoring at busy intersections.  
- Collection of actionable data through advanced analytics to inform future policy decisions.  

The solution must be scalable to accommodate up to 100 additi

In [10]:
draft_prompt = f"""
You are a senior tender documentation officer working at Dublin City Council.
You are responsible for writing detailed and high-standard tender documents in accordance with Irish and EU procurement norms.

Below are authentic excerpts from previously published public tenders related to security technologies:
{tender_texts[0][:1500]}
{tender_texts[1][:1500] if len(tender_texts) > 1 else ''}

You have also received internal notes clarifying specific needs and expectations for this procurement:
--- USER CLARIFICATIONS ---
{user_answers}

Your task is to write an extremely comprehensive **"Requirements and Specifications"** section for a new public tender to procure security cameras and video management infrastructure.

🟢 Your response MUST:
- Be at least **1500 words** in length.
- Fully reflect the **structure, level of detail, language style**, and **subsection formatting** used in Irish public tenders (like the examples above).
- Include multiple tables (e.g., camera specs, VMS features, installation steps, SLAs, etc.).
- Use **bullet points**, **numbered sections**, **section headers**, and **clear formatting**.
- Explicitly mention relevant Irish/EU laws (e.g., GDPR, CE Marking, ISO27001, 2014/24/EU).
- Present both **mandatory** and **desirable** features, clearly marked.
- Emphasize sustainability, accessibility, cybersecurity, training, and documentation expectations.
- Cover full **lifecycle**: procurement, delivery, installation, support, exit/transition.
- **Table-layout rules:** any table must fit on an A4 page in portrait orientation with 2 cm margins;
  • limit to **max 3-4 columns**, each ≤ 6 cm wide.
  • If a cell’s text would overflow, replace it with a superscript footnote symbol (e.g. ¹) and put the full explanation in a “Table Notes” paragraph directly under the table.
  • Split very wide tables into multiple stacked tables (e.g. break after the “Verification Method” column).
- **Column & text limits:** every table may contain **max 4 columns** and **max 30 characters per cell** (including spaces).
  • Overflow → superscript note mark, explanation in “Table Notes”.
  • Split if still too wide.
  • **Do NOT reuse the sample column headings; choose headings appropriate to each table.**
  Example layout (for structure only, not wording):
  | A | B | C | D¹ |
For the “Min. Numerical Value” column, always state a concrete figure and unit (e.g., “0.01 Lux”, “IP66”); **do not use Yes/No here**—if the feature is binary, move Yes/No to a separate “Supported?” column instead.
- In every table, **do NOT leave any cell blank**.
  • If the value is the same as the row above, repeat it verbatim (e.g., write “Platform” again) or use the ditto symbol “〃”.
  • Keep the column order fixed so that each row has exactly the same number of cells.

- Insert Word-friendly formatting hints (e.g. “|---|” rows exactly match the final column count) so the table can be pasted without breaking.

⚠️ Do not generate any questions. Only provide the full Requirements and Specifications section, formatted as it would appear in a formal public tender.

Start your section now.
"""


In [11]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "system", "content": "You are a senior public procurement officer writing long, formal tender documents for Irish municipalities."},
        {"role": "user", "content": draft_prompt}
    ],
    temperature=0.2,
    max_tokens=4096  # maximum allowed
)

print("📄 FINAL LONG R&S SECTION:\n")
print(response.choices[0].message.content)


📄 FINAL LONG R&S SECTION:

# **Requirements and Specifications**  
## **1. Introduction**  
Dublin City Council (hereinafter referred to as “the Contracting Authority”) invites tenders for the supply, installation, commissioning, and maintenance of a security camera system and associated video management infrastructure to enhance public safety, optimise traffic flow monitoring, and safeguard municipal assets. This procurement is conducted in accordance with the provisions of Directive 2014/24/EU, the European Union (Award of Public Authority Contracts) Regulations 2016 (S.I. No. 284 of 2016), and all other applicable Irish and EU procurement and data protection laws.  

The successful tenderer will deliver a comprehensive, scalable, and GDPR-compliant solution that meets the technical, operational, and sustainability requirements outlined herein. The solution will cover 10 public parks, 5 key intersections, and 3 municipal buildings, with a total of approximately 60 cameras to be deplo